In [1]:
import pandas as pd

In [2]:
from datetime import date
from dateutil.relativedelta import relativedelta

In [3]:
df = pd.read_csv('Inbound_Cleaned(All dup) (4).csv')
mat = pd.read_csv('MaterialMaster.csv')

In [ ]:
# merge the two dataframes on MATERIAL_NAME
# Ensure that the MATERIAL_NAME column exists in both dataframes
merged = pd.merge(df, mat, on='MATERIAL_NAME', how='left')
merged.head()

,INBOUND_DATE,PLANT_NAME,MATERIAL_NAME,NET_QUANTITY_MT,POLYMER_TYPE,SHELF_LIFE_IN_MONTH,DOWNGRADE_VALUE_LOST_PERCENT
0,2023-12-15,SINGAPORE-WAREHOUSE,MAT-0354,23.375,P-001,7,25
1,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,1.375,P-001,7,20
2,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,23.375,P-001,7,20
3,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,24.750,P-001,7,20
4,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,19.250,P-001,7,20


In [ ]:
def add_shouldbeoutbound(data_merged):
	# Ensure INBOUND_DATE is datetime
	data_merged['INBOUND_DATE'] = pd.to_datetime(data_merged['INBOUND_DATE'])

	# Calculate ShouldBeOutboundMonth row-wise
	data_merged['ShouldBeOutbound'] = data_merged.apply(
		lambda row: row['INBOUND_DATE'] + relativedelta(months=int(row['SHELF_LIFE_IN_MONTH'])),
		axis=1)

	# Convert ShouldBeOutbound to Month period
	# This will give the month of the ShouldBeOutbound date
	data_merged['ShouldBeOutboundMonth'] = data_merged['ShouldBeOutbound'].dt.to_period('M')

	# Return the modified DataFrame
	return data_merged


In [41]:
add_shouldbeoutbound(merged).head()

,INBOUND_DATE,PLANT_NAME,MATERIAL_NAME,NET_QUANTITY_MT,POLYMER_TYPE,SHELF_LIFE_IN_MONTH,DOWNGRADE_VALUE_LOST_PERCENT,ShouldBeOutbound,ShouldBeOutboundMonth
0,2023-12-15,SINGAPORE-WAREHOUSE,MAT-0354,23.375,P-001,7,25,2024-07-15,2024-07
1,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,1.375,P-001,7,20,2024-07-22,2024-07
2,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,23.375,P-001,7,20,2024-07-22,2024-07
3,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,24.750,P-001,7,20,2024-07-22,2024-07
4,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,19.250,P-001,7,20,2024-07-22,2024-07


In [38]:
merged.sample(10)

,INBOUND_DATE,PLANT_NAME,MATERIAL_NAME,NET_QUANTITY_MT,POLYMER_TYPE,SHELF_LIFE_IN_MONTH,DOWNGRADE_VALUE_LOST_PERCENT,ShouldBeOutbound
5913,2024-10-21,CHINA-WAREHOUSE,MAT-0087,24.750,P-001,3,20,2025-01-21
1356,2024-05-19,SINGAPORE-WAREHOUSE,MAT-0309,20.625,P-001,5,15,2024-10-19
1899,2024-08-13,SINGAPORE-WAREHOUSE,MAT-0234,13.750,P-001,5,10,2025-01-13
3479,2024-03-03,CHINA-WAREHOUSE,MAT-0112,1.375,P-001,6,20,2024-09-03
2719,2023-12-31,CHINA-WAREHOUSE,MAT-0319,24.750,P-001,8,5,2024-08-31
438,2024-02-02,SINGAPORE-WAREHOUSE,MAT-0104,24.750,P-001,6,15,2024-08-02
202,2023-12-15,SINGAPORE-WAREHOUSE,MAT-0099,5.500,P-001,6,20,2024-06-15
564,2024-02-25,SINGAPORE-WAREHOUSE,MAT-0273,19.250,P-001,8,30,2024-10-25
3361,2024-02-09,CHINA-WAREHOUSE,MAT-0196,5.000,P-002,4,35,2024-06-09
4769,2024-05-27,CHINA-WAREHOUSE,MAT-0007,24.750,P-001,2,10,2024-07-27


In [7]:
outbound = pd.read_csv('outbound_clean.csv')
operation = pd.read_csv('OperationCost.csv')

In [8]:
outbound.head()

,OUTBOUND_DATE,PLANT_NAME,MODE_OF_TRANSPORT,MATERIAL_NAME,CUSTOMER_NUMBER,NET_QUANTITY_MT
0,2024/01/02,CHINA-WAREHOUSE,Truck,MAT-0013,CST-00001,25.50
1,2024/01/02,CHINA-WAREHOUSE,Truck,MAT-0268,CST-00002,25.50
2,2024/01/02,CHINA-WAREHOUSE,Truck,MAT-0145,CST-00003,25.50
3,2024/01/02,CHINA-WAREHOUSE,Truck,MAT-0428,CST-00004,24.75
4,2024/01/02,CHINA-WAREHOUSE,Truck,MAT-0264,CST-00005,24.75


In [ ]:
# Groupby  PLANT_NAME OUTBOUND_DATE  MODE_OF_TRANSPORT MATERIAL_NAME CUSTOMER_NUMBER and sum NET_QUANTITY_MT
outbound_group = outbound.groupby(
    ['PLANT_NAME', 'OUTBOUND_DATE', 'MODE_OF_TRANSPORT', 'MATERIAL_NAME', 'CUSTOMER_NUMBER']
)['NET_QUANTITY_MT'].sum().reset_index()



22179

In [10]:
operation.head()

,Operation,Plant/Mode of Transport,Cost,Currency
0,Inventory Storage per MT per day,SINGAPORE WAREHOUSE,15.0,SGD
1,Inventory Storage per MT per day,CHINA WAREHOUSE,1.7,CNY
2,Transfer cost per container (24.75MT),Truck,150.0,SGD
3,Transfer cost per container (24.75MT),Marine,75.0,USD


### Check how many row that NET_QUANTITY_MT more than 24.75

In [ ]:
# Check how many row that NET_QUANTITY_MT more than 24.75
# outbound[outbound['NET_QUANTITY_MT'] > 24.75].shape[0]
len(outbound)

In [11]:
transfer = operation[operation['Operation'] == 'Transfer cost per container (24.75MT)']

In [12]:
transfer

,Operation,Plant/Mode of Transport,Cost,Currency
2,Transfer cost per container (24.75MT),Truck,150.0,SGD
3,Transfer cost per container (24.75MT),Marine,75.0,USD


In [13]:
# calculate transfer cost
def calculate_transfer_cost(row):
    # Check how many trasnport we need to use from NET_QUANTITY_MT / 24.75 example 26 MT will need 2
    if row['NET_QUANTITY_MT'] % 24.75 == 0:
        num_transports = row['NET_QUANTITY_MT'] // 24.75
    else:
        num_transports = row['NET_QUANTITY_MT'] // 24.75 + 1

    # Get the cost based on MODE_OF_TRANSPORT
    if row['MODE_OF_TRANSPORT'] == 'Truck':
        cost = num_transports * transfer['Cost'].values[0]
    elif row['MODE_OF_TRANSPORT'] == 'Marine':
        cost = num_transports * transfer['Cost'].values[1]

    # Calculate SGD and CNY to USD
    if row['PLANT_NAME'] == 'CHINA-WAREHOUSE':
        cost =  cost / 7.85  # Example conversion rate
    elif row['PLANT_NAME'] == 'SINGAPORE-WAREHOUSE':
        cost =  cost * 0.14  # Example conversion rate
    
    return round(cost, 3)

In [14]:
outbound_group['Transfer_Cost'] = outbound_group.apply(calculate_transfer_cost, axis=1)

In [15]:
outbound_group_merge = pd.merge(outbound_group, mat, on='MATERIAL_NAME', how='left')

In [16]:
outbound_group.head()

,PLANT_NAME,OUTBOUND_DATE,MODE_OF_TRANSPORT,MATERIAL_NAME,CUSTOMER_NUMBER,NET_QUANTITY_MT,Transfer_Cost
0,CHINA-WAREHOUSE,2024/01/02,Truck,MAT-0007,CST-00023,24.76,38.217
1,CHINA-WAREHOUSE,2024/01/02,Truck,MAT-0013,CST-00001,25.50,38.217
2,CHINA-WAREHOUSE,2024/01/02,Truck,MAT-0013,CST-00031,3.00,19.108
3,CHINA-WAREHOUSE,2024/01/02,Truck,MAT-0015,CST-00013,24.75,19.108
4,CHINA-WAREHOUSE,2024/01/02,Truck,MAT-0015,CST-00014,24.75,19.108


In [17]:
outbound_group_merge.head()

,PLANT_NAME,OUTBOUND_DATE,MODE_OF_TRANSPORT,MATERIAL_NAME,CUSTOMER_NUMBER,NET_QUANTITY_MT,Transfer_Cost,POLYMER_TYPE,SHELF_LIFE_IN_MONTH,DOWNGRADE_VALUE_LOST_PERCENT
0,CHINA-WAREHOUSE,2024/01/02,Truck,MAT-0007,CST-00023,24.76,38.217,P-001,2,10
1,CHINA-WAREHOUSE,2024/01/02,Truck,MAT-0013,CST-00001,25.50,38.217,P-002,3,25
2,CHINA-WAREHOUSE,2024/01/02,Truck,MAT-0013,CST-00031,3.00,19.108,P-002,3,25
3,CHINA-WAREHOUSE,2024/01/02,Truck,MAT-0015,CST-00013,24.75,19.108,P-001,8,20
4,CHINA-WAREHOUSE,2024/01/02,Truck,MAT-0015,CST-00014,24.75,19.108,P-001,8,20


In [18]:
outbound_group.to_csv('outbound_group_withcost.csv', index=False)

In [19]:
inbound = pd.read_csv('Inbound_Cleaned(All dup) (4).csv')
# get month from INBOUND_DATE from inbound
inbound['INBOUND_DATE'] = pd.to_datetime(inbound['INBOUND_DATE'])
inbound['INBOUND_MONTH'] = inbound['INBOUND_DATE'].dt.month
inbound['INBOUND_YEAR'] = inbound['INBOUND_DATE'].dt.year

In [20]:
inbound

,INBOUND_DATE,PLANT_NAME,MATERIAL_NAME,NET_QUANTITY_MT,INBOUND_MONTH,INBOUND_YEAR
0,2023-12-15,SINGAPORE-WAREHOUSE,MAT-0354,23.375,12,2023
1,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,1.375,12,2023
2,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,23.375,12,2023
3,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,24.750,12,2023
4,2023-12-22,SINGAPORE-WAREHOUSE,MAT-0413,19.250,12,2023
...,...,...,...,...,...,...
6445,2024-12-22,CHINA-WAREHOUSE,MAT-0339,25.500,12,2024
6446,2024-12-24,CHINA-WAREHOUSE,MAT-0265,24.750,12,2024
6447,2024-12-24,CHINA-WAREHOUSE,MAT-0042,24.750,12,2024
6448,2024-12-24,CHINA-WAREHOUSE,MAT-0232,24.750,12,2024


In [21]:
# Groupby INBOUND_MONTH only and sum NET_QUANTITY_MT
inbound_group = inbound.groupby(['INBOUND_YEAR','INBOUND_MONTH'])['NET_QUANTITY_MT'].sum().reset_index()
inbound_group['NET_QUANTITY_KT'] = round(inbound_group['NET_QUANTITY_MT'] / 1000,3)
inbound_group


,INBOUND_YEAR,INBOUND_MONTH,NET_QUANTITY_MT,NET_QUANTITY_KT
0,2023,11,2070.915,2.071
1,2023,12,6249.175,6.249
2,2024,1,7417.125,7.417
3,2024,2,6425.926,6.426
4,2024,3,11515.263,11.515
5,2024,4,8655.925,8.656
6,2024,5,10996.603,10.997
7,2024,6,8059.525,8.060
8,2024,7,7877.959,7.878
9,2024,8,9739.409,9.739


In [22]:
inbound_group.to_csv('inbound_group.csv', index=False)

In [ ]:
outbound = pd.read_csv('outbound_clean.csv')
# get month from OUTBOUND_DATE from outbound
outbound['OUTBOUND_DATE'] = pd.to_datetime(outbound['OUTBOUND_DATE'])
outbound['OUTBOUND_MONTH'] = outbound['OUTBOUND_DATE'].dt.month
outbound['OUTBOUND_YEAR'] = outbound['OUTBOUND_DATE'].dt.year

# Groupby OUTBOUND_MONTH only and sum NET_QUANTITY_MT
outbound_group = outbound.groupby(['OUTBOUND_YEAR','OUTBOUND_MONTH'])['NET_QUANTITY_MT'].sum().reset_index()
outbound_group['NET_QUANTITY_KT'] = round(outbound_group['NET_QUANTITY_MT'] / 1000,3)
outbound_group




,OUTBOUND_YEAR,OUTBOUND_MONTH,NET_QUANTITY_MT,NET_QUANTITY_KT
0,2023,12,119.625,0.120
1,2024,1,25674.885,25.675
2,2024,2,16321.775,16.322
3,2024,3,23532.305,23.532
4,2024,4,23166.160,23.166
5,2024,5,25495.910,25.496
6,2024,6,25371.840,25.372
7,2024,7,26473.480,26.473
8,2024,8,25489.110,25.489
9,2024,9,30131.255,30.131


In [72]:
outbound = pd.read_csv('outbound_clean.csv')
# merge outbound and material master
outbound['OUTBOUND_DATE'] = pd.to_datetime(outbound['OUTBOUND_DATE'])
outbound['OUTBOUND_MONTH'] = outbound['OUTBOUND_DATE'].dt.month
outbound['OUTBOUND_YEAR'] = outbound['OUTBOUND_DATE'].dt.year
outbound_mat = pd.merge(outbound, mat, on='MATERIAL_NAME', how='left')

# CHECK IS HOLIDAY
chaina_holiday_str = [
    '01/01', '01/28', '01/29', '01/30', '01/31', '02/01', '02/02', '02/03', '02/04',
    '04/04', '04/05', '04/06',
    '05/01', '05/02', '05/03', '05/04', '05/05',
    '06/02',
    '10/01', '10/02', '10/03', '10/04', '10/05', '10/06', '10/07', '10/08'
]

SGP_holiday_str = [
    '01/01',
    '02/10', '02/11', '02/12',
    '03/29',
    '04/10',
    '05/01',
    '05/22',
    '06/17',
    '08/09',
    '10/31',
    '12/25'
]
# Function to check if a date is inbetween a holiday 2 days by PLANT_NAME and date string
def is_holiday(row):
    # Get the correct holiday list based on PLANT_NAME
    if row['PLANT_NAME'] == 'CHINA-WAREHOUSE':
        holiday_list = chaina_holiday_str
    elif row['PLANT_NAME'] == 'SINGAPORE-WAREHOUSE':
        holiday_list = SGP_holiday_str
    else:
        return False

    # Get the date for checking
    date = row['OUTBOUND_DATE']
    # Check the day before, the day of, and the day after
    for offset in [-1, 0, 1]:
        check_date = (date + pd.Timedelta(days=offset)).strftime('%m/%d')
        if check_date in holiday_list:
            return True
    return False
    
# Apply the function to create a new column 'IS_HOLIDAY'
outbound_mat['IS_HOLIDAY'] = outbound_mat.apply(is_holiday, axis=1)
outbound_mat['IS_HOLIDAY'] = outbound_mat['IS_HOLIDAY'].astype(int)
# get row that IS_HOLIDAY is 1
outbound_mat_holiday = outbound_mat[outbound_mat['IS_HOLIDAY'] == 1]
# get row that IS_HOLIDAY is 0
outbound_mat_no_holiday = outbound_mat[outbound_mat['IS_HOLIDAY'] == 0]
# save outbound_mat to csv
outbound_mat.to_csv('outbound_mat_holiday.csv', index=False)


In [74]:
# Gruopby Week and sum NET_QUANTITY_KT
outbound_mat['OUTBOUND_WEEK'] = outbound_mat['OUTBOUND_DATE'].dt.isocalendar().week
outbound_week_group = outbound_mat.groupby(['OUTBOUND_YEAR', 'OUTBOUND_WEEK','PLANT_NAME','MODE_OF_TRANSPORT','CUSTOMER_NUMBER','MATERIAL_NAME','POLYMER_TYPE','SHELF_LIFE_IN_MONTH','IS_HOLIDAY'])['NET_QUANTITY_MT'].sum().reset_index()
# outbound_week_group['NET_QUANTITY_KT'] = round(outbound_week_group['NET_QUANTITY_MT'] / 1000, 3)
outbound_week_group.head()

,OUTBOUND_YEAR,OUTBOUND_WEEK,PLANT_NAME,MODE_OF_TRANSPORT,CUSTOMER_NUMBER,MATERIAL_NAME,POLYMER_TYPE,SHELF_LIFE_IN_MONTH,IS_HOLIDAY,NET_QUANTITY_MT
0,2023,49,SINGAPORE-WAREHOUSE,Marine,CST-01001,MAT-0050,P-001,6,0,49.500
1,2023,52,SINGAPORE-WAREHOUSE,Marine,CST-01002,MAT-0056,P-001,3,1,49.500
2,2023,52,SINGAPORE-WAREHOUSE,Marine,CST-01003,MAT-0210,P-001,3,1,6.875
3,2023,52,SINGAPORE-WAREHOUSE,Marine,CST-01003,MAT-0303,P-001,5,1,2.500
4,2023,52,SINGAPORE-WAREHOUSE,Marine,CST-01003,MAT-0345,P-001,6,1,1.250


In [56]:
outbound_week_group

,OUTBOUND_YEAR,OUTBOUND_WEEK,NET_QUANTITY_MT,NET_QUANTITY_KT
0,2023,49,49.500,0.050
1,2023,52,70.125,0.070
2,2024,1,6637.430,6.637
3,2024,2,6709.720,6.710
4,2024,3,5721.265,5.721
5,2024,4,5721.065,5.721
6,2024,5,4133.965,4.134
7,2024,6,2718.775,2.719
8,2024,7,1855.725,1.856
9,2024,8,5962.190,5.962


In [75]:
outbound_week_group.to_csv('outbound_mat_holiday_byweek.csv', index=False)